In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re
import string

In [2]:
fake_news_path = "C:\\Users\\89tom\\OneDrive\\Desktop\\Pace\\Forth Semester\\DataScienceCS675\\Project\\fake_news\\Fake.csv" 
actual_news_path = "C:\\Users\\89tom\\OneDrive\\Desktop\\Pace\\Forth Semester\\DataScienceCS675\\Project\\fake_news\\True.csv"

In [3]:
# importing fake news dataset
df_fake = pd.read_csv(fake_news_path)
#importing actual news
df_actual = pd.read_csv(actual_news_path)


In [4]:
df_fake['classification'] = 1
df_actual['classification'] = 0
#combining both datasets
df = pd.concat([df_fake,df_actual],axis=0)
df.head()

,title,text,subject,date,classification
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [5]:
#Shuffling all the rows randomly
df = df.sample(frac = 1).reset_index(drop=True)
df.head()

,title,text,subject,date,classification
0,Trump’s Excuse For All His Sexist Statements ...,In case you ve been living under a rock for yo...,News,"March 28, 2016",1
1,Mexican police arrest suspect in crime journal...,MEXICO CITY (Reuters) - A man alleged to be be...,worldnews,"December 26, 2017",0
2,U.S. approves license for Ukraine to buy small...,WASHINGTON (Reuters) - The U.S. State Departme...,worldnews,"December 21, 2017",0
3,"U.S. governors, hackers, academics team up to ...",(Reuters) - Hackers are joining forces with U....,politicsNews,"October 9, 2017",0
4,Five Facts About The Obama Economy That Trump...,Trump likes to pretend he was handed a mess ...,News,"March 3, 2017",1


In [6]:
#merging both title and text columns
df['content'] = df['title'] + df['text']
df.isnull().sum()

title             0
text              0
subject           0
date              0
classification    0
content           0
dtype: int64

In [7]:
# Removing special characters from dataset
def replace_text(text):
    new_text = text.lower()
    new_text = re.sub('\[.*?\]','',new_text)
    new_text = re.sub("\\W","",new_text)
    new_text = re.sub('https?://\S+|www\.\S+','',new_text)
    new_text = re.sub('<.*?>+','',new_text)
    new_text = re.sub('\n','',new_text)
    new_text = re.sub('\w*\d\w*','',new_text)
    new_text = re.sub('[%s]'% re.escape(string.punctuation),'',new_text)
    return new_text

In [8]:
df['content'] = df['content'].apply(replace_text)
X = df['content']
y = df['classification']

# vectorizing the data
vectorization = TfidfVectorizer()
X_vector = vectorization.fit_transform(X)

X_train, X_test, y_train,y_test = train_test_split(X_vector,y,test_size=0.25,random_state=42)

# fitting the data to the model
model = LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.01      0.02      5390
           1       0.52      1.00      0.69      5835

    accuracy                           0.52     11225
   macro avg       0.76      0.50      0.35     11225
weighted avg       0.75      0.52      0.37     11225

